### 자동차 등록 자료 통계 xlsx -> 필요부문 추출 후csv 변환

In [5]:
# 라이브러리 추가
import os
import pandas as pd

### data 중, 자동차 등록 자료 통계 xlsx 만 선별후 한글 이름 변경

In [6]:
path = "./data"
file_list = os.listdir(path)
car_file_list = [file for file in file_list if file.endswith('자동차_등록자료_통계.xlsx')]

In [7]:
for car_file in car_file_list:
    src = os.path.join(path, car_file)
    chg_name = car_file.split('_')[0][0:4] + '_' + car_file.split('_')[1][0:-1] + '_' + 'Car_Info' + '.xlsx'
    dst = os.path.join(path, chg_name)
    os.rename(src, dst)

In [8]:
file_list = os.listdir(path)
car_file_list = [file for file in file_list if file.endswith('Car_Info.xlsx')]

In [9]:
print(car_file_list)

['2020_01_Car_Info.xlsx', '2020_02_Car_Info.xlsx', '2020_03_Car_Info.xlsx', '2020_04_Car_Info.xlsx', '2020_05_Car_Info.xlsx', '2020_06_Car_Info.xlsx', '2020_07_Car_Info.xlsx', '2020_08_Car_Info.xlsx', '2020_09_Car_Info.xlsx', '2020_10_Car_Info.xlsx']


### excel 파일을 읽기 위해 작업 실시

!pip3 install xlrd
!pip3 install openpyxl

### 1. 전처리 작업 실시

In [128]:
# 각 파일을 1개씩 불러와 전처리 실시 
count = 0
for car_file in car_file_list:
    
    target_file = os.path.join(path, car_file)
    df = pd.read_excel(target_file, engine='openpyxl', sheet_name="10.연료별_등록현황", header = 2)
    df = df.rename(columns={'Unnamed: 2' : '합계'}) # 합계의 경우, 해당 줄에 컬럼명이 없어 추가 함 
    
    if count == 0:
        tmp_df = pd.DataFrame(columns=df.columns)    # 최종 전처리 프레임
    
    # 소계인 데이터만 추출 (업무용 / 비업무용 등은 제거)
    for i in range(1, len(df) - 17, 17):
        tmp = df[i:i+17].fillna(method='pad')[-1:]
        tmp_df = pd.concat([tmp_df, tmp])
        
    # 소계 중 한대도 없는 차량의 경우 제거 
    tmp_df = tmp_df[tmp_df['계'] != 0.0]
    
    # 불필요한 행 제거 
    tmp_df = tmp_df.drop('Unnamed: 21', axis=1)
    
    # 친환경 차량 구분을 위한 항목 추가 
    tmp_lst = list()
    for tmp in tmp_df.연료별:
        if (tmp.find('수소') != -1 ) | (tmp.find('전기') != -1):
            tmp_lst.append('친환경')
        else : 
            tmp_lst.append('비친환경')

    tmp_df['친환경'] = tmp_lst
    
    count += 1

#csv 형태로 변환 저장 [한글 깨짐 방지를 위해 utf-8 설정]
tmp_df.to_csv(target_file.replace('.xlsx', '.csv'),mode = 'w',encoding='utf-8-sig')